# Tractorun

This notebook demonstrates the `Tractorun` library and CLI tool for running distributed machine learning tasks on the `Tracto`. `Tractorun` provides convenient tools to integrate with Tracto distributed data processing system, enabling execution and management of machine learning training jobs.

Tractorun:
1. Manages the configuration and coordination of distributed training.
2. Provides tools for working with `YtDataset` (allows you to use data on Tracto as a dataset), checkpoints, saving models, interacting with `tensorproxy`, and more.
3. Ensures integration with the Tracto ecosystem.

In this notebook, we cover the following steps:

1. Uploading a PyTorch dataset to Tracto.
2. Training a model using MNIST. We perform model training on the MNIST dataset directly from a Jupyter Notebook, leveraging Tracto as the computation platform.
3. Running the same training with Tractorun CLI. We'll demonstrate how to run the same training job via the command line using the Tractorun CLI.

We use the official PyTorch [MNIST training example](https://github.com/pytorch/examples/blob/cdef4d43fb1a2c6c4349daa5080e4e8731c34569/mnist/main.py) as a reference and show how to modify it with minimal changes to run using `Tractorun`.

In [1]:
from yt import wrapper as yt
from yt import type_info

In [2]:
import uuid
import sys
import io

## Create a base directory for examples

In [4]:
# configure environment to run this notebooks
import uuid
import yt.wrapper as yt

username = yt.get_user_name()
if yt.exists(f"//sys/users/{username}/@user_info/home_path"):
    # prepare working directory on distributed file system
    user_info = yt.get(f"//sys/users/{yt.get_user_name()}/@user_info")
    homedir = user_info["home_path"]
    # find avaliable vm presets
    cpu_pool_trees = [pool_tree for pool_tree in user_info["available_pool_trees"] if pool_tree.endswith("cpu")] or ["default"]
    h100_pool_trees = [pool_tree for pool_tree in user_info["available_pool_trees"] if pool_tree.endswith("h100")]
    h100_8_pool_trees = [pool_tree for pool_tree in user_info["available_pool_trees"] if pool_tree.endswith("h100-8")]
    workdir = f"{homedir}/tmp/demo_workdir/{uuid.uuid4().hex}"
else:
    cpu_pool_trees = ["default"]
    h100_pool_trees = ["gpu_h100"]
    h100_8_pool_trees = ["gpu_h100"]
    workdir = f"//tmp/examples/{uuid.uuid4().hex}"

yt.create("map_node", workdir, recursive=True, ignore_existing=True)
print("Current working directory:", workdir)

Current working directory: //home/equal_amethyst_vulture/tmp/demo_workdir/594e35db3a434a748e844f4d22c07e6c


## Ensure torch and torchvision exist

Let's ensure that the system has installed `torch` and `torchvision`.

In [6]:
import torch
import torchvision 
from torchvision import datasets, transforms

## Upload dataset to Tracto

For this demonstration, we will use the MNIST dataset from the `torchvision` library and upload it to Tracto. Some rows in the dataset exceed the standard limits, so we will set `table_writer={"max_row_weight": 50 * 1024 * 1024}`.

In [8]:
# https://github.com/pytorch/examples/blob/26de41904319c7094afc53a3ee809de47112d387/mnist/main.py#L119
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,)),
    ],
)

dataset_train_local = datasets.MNIST("./mnist", train=True, download=True)
dataset_test_local = datasets.MNIST("./mnist", train=False, download=True)

  0%|          | 0.00/9.91M [00:00<?, ?B/s]

  1%|          | 98.3k/9.91M [00:00<00:19, 514kB/s]

  2%|▏         | 197k/9.91M [00:00<00:18, 518kB/s] 

  4%|▍         | 426k/9.91M [00:00<00:09, 1.04MB/s]

  9%|▉         | 918k/9.91M [00:00<00:04, 2.21MB/s]

...

 26%|██▌       | 426k/1.65M [00:00<00:01, 991kB/s]

 56%|█████▌    | 918k/1.65M [00:00<00:00, 2.10MB/s]

100%|██████████| 1.65M/1.65M [00:00<00:00, 2.40MB/s]

  0%|          | 0.00/4.54k [00:00<?, ?B/s]

100%|██████████| 4.54k/4.54k [00:00<00:00, 8.62MB/s]

Let's upload on Tracto the MNIST dataset as tensors and as simple types. There are 4 columns:
* `image` - raw png image. This column has the tag "image/png" which allows to draw images directly in the Tracto UI.
* `number` - human-readable label.
* `data` and `labels` - serialized tensor form of dataset's data and label.

It is more efficient to save ready-to-use tensors in YT right away to save time and resources during model training. In the following examples, we will work only with columns containing tensors.

In [10]:
schema = yt.schema.TableSchema()
schema.add_column("image", type_info.Tagged[type_info.String, "image/png"])
schema.add_column("number", type_info.Int8)
schema.add_column("data", type_info.String)
schema.add_column("labels", type_info.String)

TableSchema({'value': [{'name': 'image', 'type_v3': {'type_name': 'tagged', 'item': 'string', 'tag': 'image/png'}}, {'name': 'number', 'type_v3': 'int8'}, {'name': 'data', 'type_v3': 'string'}, {'name': 'labels', 'type_v3': 'string'}], 'attributes': {'strict': True, 'unique_keys': False}})

In [11]:
from tractorun.backend.tractorch import TensorSerializer

dataset_train_path = f"{workdir}/dataset_train"
dataset_test_path = f"{workdir}/dataset_test"
print(dataset_train_path)
print(dataset_test_path)

yt.create("table", dataset_train_path, force=True, attributes={"schema": schema.to_yson_type()})
yt.create("table", dataset_test_path, force=True, attributes={"schema": schema.to_yson_type()})

def pil_to_png(image):
    r = io.BytesIO()
    image.save(r, format="PNG")
    return r.getvalue()

ts = TensorSerializer()

yt_train_data = [
    {
        "image": pil_to_png(data),
        "number": labels,
        "labels": ts.serialize(labels),
        "data": ts.serialize(transform(data)),
    }
    for data, labels in dataset_train_local
]
yt.write_table(dataset_train_path, yt_train_data, table_writer={"max_row_weight": 50 * 1024 * 1024})

yt_test_data = [
    {
        "image": pil_to_png(data),
        "number": labels,
        "labels": ts.serialize(labels),
        "data": ts.serialize(transform(data)),
    }
    for data, labels in dataset_test_local
]
yt.write_table(dataset_test_path, yt_test_data, table_writer={"max_row_weight": 50 * 1024 * 1024})

//home/equal_amethyst_vulture/tmp/demo_workdir/594e35db3a434a748e844f4d22c07e6c/dataset_train
//home/equal_amethyst_vulture/tmp/demo_workdir/594e35db3a434a748e844f4d22c07e6c/dataset_test


## Run training

Tractorun store some data to the training dir:
1. Checkpoints.
2. Metadata about each training run.
3. Models.
4. Some locks.
5. etc

Let's create and cleanup the training dir.

In [13]:
training_dir = f"{workdir}/tractorun"
yt.create("map_node", training_dir, force=True)

print(training_dir)

//home/equal_amethyst_vulture/tmp/demo_workdir/594e35db3a434a748e844f4d22c07e6c/tractorun


The model training process run in a Docker container. When launching from a Jupyter Notebook, it is important to ensure that the same container as in the `Kernel` is used.

We use the official PyTorch [MNIST training example](https://github.com/pytorch/examples/blob/cdef4d43fb1a2c6c4349daa5080e4e8731c34569/mnist/main.py) as a reference and show how to modify it with minimal changes to run using Tractorun:
1. Add `toolbox: Toolbox` to the main function. Toolbox object provides useful utils for training like checkpoint manager, coordination metadata, initialized ytsaurus client, and more.
2. Add `file=sys.stderr` to each print.
3. Use `YtTensorDataset` instead of default `torch.Dataset`.
4. Call magic function `tractorun.run.run`.

<details>
  <summary>Show the full diff</summary>

```diff
@@ -6,6 +6,13 @@
 from torchvision import datasets, transforms
 from torch.optim.lr_scheduler import StepLR

+from tractorun.backend.tractorch import YtTensorDataset, Tractorch
+from tractorun.toolbox import Toolbox
+from tractorun.run import run
+from tractorun.mesh import Mesh
+from tractorun.resources import Resources
+from tractorun.stderr_reader import StderrMode
+from tractorun.backend.tractorch.serializer import TensorSerializer

 class Net(nn.Module):
     def __init__(self):
@@ -45,7 +52,7 @@
         if batch_idx % args.log_interval == 0:
             print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                 epoch, batch_idx * len(data), len(train_loader.dataset),
-                100. * batch_idx / len(train_loader), loss.item()))
+                100. * batch_idx / len(train_loader), loss.item()), file=sys.stderr)
             if args.dry_run:
                 break

@@ -66,10 +73,10 @@

     print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
         test_loss, correct, len(test_loader.dataset),
-        100. * correct / len(test_loader.dataset)))
+        100. * correct / len(test_loader.dataset)), file=sys.stderr)


-def main():
+def main(toolbox: Toolbox):
     # Training settings
     parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
     parser.add_argument('--batch-size', type=int, default=64, metavar='N',
@@ -94,7 +101,7 @@
                         help='how many batches to wait before logging training status')
     parser.add_argument('--save-model', action='store_true', default=False,
                         help='For Saving the current Model')
-    args = parser.parse_args()
+    args = parser.parse_args([])
     use_cuda = not args.no_cuda and torch.cuda.is_available()
     use_mps = not args.no_mps and torch.backends.mps.is_available()

@@ -120,10 +127,9 @@
         transforms.ToTensor(),
         transforms.Normalize((0.1307,), (0.3081,))
         ])
-    dataset1 = datasets.MNIST('../data', train=True, download=True,
-                       transform=transform)
-    dataset2 = datasets.MNIST('../data', train=False,
-                       transform=transform)
+    dataset1 = YtTensorDataset(path=dataset_train_path, columns=['data', 'labels'])
+    dataset2 = YtTensorDataset(path=dataset_test_path, columns=['data', 'labels'])
+
     train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
     test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

@@ -137,9 +143,20 @@
         scheduler.step()

     if args.save_model:
-        torch.save(model.state_dict(), "mnist_cnn.pt")
+        ts = TensorSerializer()
+        toolbox.save_model(ts.serialize(model.state_dict()), dataset_train_path, metadata={})


-if __name__ == '__main__':
-    main()
+run(
+    main,
+    backend=Tractorch(),
+    yt_path=training_dir,
+    mesh=Mesh(node_count=1, process_per_node=1, gpu_per_process=1, pool_trees=h100_pool_trees),
+    resources=Resources(
+        cpu_limit=8,
+        memory_limit=105899345920,
+    ),
+    proxy_stderr_mode=StderrMode.primary,
+)
```
</details>

<font color="red">IMPORTANT NOTE</font> In this example we are running tractorun directly from Jupyter notebook.

This is a convenient method for experiments and demonstrations, as tractorun uses [pickle](https://docs.python.org/3/library/pickle.html) for easy serialization of the entire notebook state and transferring it to the cluster. This means that all variables will be available in the model training function, and tractorun will attempt to transfer all Python modules from the local environment to the cluster.

However, this method does not ensure reproducibility of the run of model's training. For production processes, use the execution via the tractorun CLI, which is described below.

In [16]:
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

from tractorun.backend.tractorch import YtTensorDataset, Tractorch
from tractorun.toolbox import Toolbox
from tractorun.run import run
from tractorun.mesh import Mesh
from tractorun.resources import Resources
from tractorun.stderr_reader import StderrMode
from tractorun.backend.tractorch.serializer import TensorSerializer


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output


def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()), file=sys.stderr)
            if args.dry_run:
                break


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)), file=sys.stderr)


def main(toolbox: Toolbox):
    # Training settings
    parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
    parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                        help='input batch size for training (default: 64)')
    parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
                        help='input batch size for testing (default: 1000)')
    parser.add_argument('--epochs', type=int, default=14, metavar='N',
                        help='number of epochs to train (default: 14)')
    parser.add_argument('--lr', type=float, default=1.0, metavar='LR',
                        help='learning rate (default: 1.0)')
    parser.add_argument('--gamma', type=float, default=0.7, metavar='M',
                        help='Learning rate step gamma (default: 0.7)')
    parser.add_argument('--no-cuda', action='store_true', default=False,
                        help='disables CUDA training')
    parser.add_argument('--no-mps', action='store_true', default=False,
                        help='disables macOS GPU training')
    parser.add_argument('--dry-run', action='store_true', default=False,
                        help='quickly check a single pass')
    parser.add_argument('--seed', type=int, default=1, metavar='S',
                        help='random seed (default: 1)')
    parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                        help='how many batches to wait before logging training status')
    parser.add_argument('--save-model', action='store_true', default=False,
                        help='For Saving the current Model')
    args = parser.parse_args([])
    use_cuda = not args.no_cuda and torch.cuda.is_available()
    use_mps = not args.no_mps and torch.backends.mps.is_available()

    torch.manual_seed(args.seed)

    if use_cuda:
        device = torch.device("cuda")
    elif use_mps:
        device = torch.device("mps")
    else:
        device = torch.device("cpu")

    train_kwargs = {'batch_size': args.batch_size}
    test_kwargs = {'batch_size': args.test_batch_size}
    if use_cuda:
        cuda_kwargs = {'num_workers': 1,
                       'pin_memory': True,
                       'shuffle': False}
        train_kwargs.update(cuda_kwargs)
        test_kwargs.update(cuda_kwargs)

    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])
    dataset1 = YtTensorDataset(path=dataset_train_path, yt_client=toolbox.yt_client, columns=['data', 'labels'])
    dataset2 = YtTensorDataset(path=dataset_test_path, yt_client=toolbox.yt_client, columns=['data', 'labels'])

    train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
    test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

    model = Net().to(device)
    optimizer = optim.Adadelta(model.parameters(), lr=args.lr)

    scheduler = StepLR(optimizer, step_size=1, gamma=args.gamma)
    for epoch in range(1, args.epochs + 1):
        train(args, model, device, train_loader, optimizer, epoch)
        test(model, device, test_loader)
        scheduler.step()

    if args.save_model:
        ts = TensorSerializer()
        toolbox.save_model(ts.serialize(model.state_dict()), dataset_train_path, metadata={})


run(
    main,
    backend=Tractorch(),
    yt_path=training_dir,
    mesh=Mesh(node_count=1, process_per_node=1, gpu_per_process=1, pool_trees=h100_pool_trees),
    resources=Resources(
        cpu_limit=10,
        memory_limit=32212254720,
    ),
    proxy_stderr_mode=StderrMode.primary,
)

2025-06-19 20:44:24,979	WARNING	Cannot locate file of the module (__name__: torch.ops, __file__: _ops.py)


2025-06-19 20:44:24,981	WARNING	Cannot locate file of the module (__name__: torch.classes, __file__: _classes.py)


2025-06-19 20:44:27,225	INFO	Operation started: https://playground.tracto.ai/playground/operations/833c9385-39d5f04a-24dd03e8-b3861697/details


2025-06-19 20:44:27,268	INFO	( 0 min) operation 833c9385-39d5f04a-24dd03e8-b3861697 initializing


2025-06-19 20:44:30,066	INFO	( 0 min) Unrecognized spec: {'enable_partitioned_data_balancing': false}


...

Train Epoch: 14 [2560/60000 (4%)]	Loss: 0.002235
Train Epoch: 14 [3200/60000 (5%)]	Loss: 0.006807
Train Epoch: 14 [3840/60000 (6%)]	Loss: 0.003513
Train Epoch: 14 [4480/60000 (7%)]	Loss: 0.004041
Train Epoch: 14 [5120/60000 (9%)]	Loss: 0.046514
Train Epoch: 14 [5760/60000 (10%)]	Loss: 0.030836
Train Epoch: 14 [6400/60000 (11%)]	Loss: 0.138029
Train Epoch: 14 [7040/60000 (12%)]	Loss: 0.138985
Train Epoch: 14 [7680/60000 (13%)]	Loss: 0.005225
Train Epoch: 14 [8320/60000 (14%)]	Loss: 0.003691
Train Epoch: 14 [8960/60000 (15%)]	Loss: 0.043210
Train Epoch: 14 [9600/60000 (16%)]	Loss: 0.007291
Train Epoch: 14 [10240/60000 (17%)]	Loss: 0.068542
Train Epoch: 14 [10880/60000 (18%)]	Loss: 0.001177
Train Epoch: 14 [11520/60000 (19%)]	Loss: 0.013633
Train Epoch: 14 [12160/60000 (20%)]	Loss: 0.020149
Train Epoch: 14 [12800/60000 (21%)]	Loss: 0.022902
Train Epoch: 14 [13440/60000 (22%)]	Loss: 0.000106
Train Epoch: 14 [14080/60000 (23%)]	Loss: 0.019971
Train Epoch: 14 [14720/60000 (25%)]	Loss: 0.0142

Train Epoch: 14 [28800/60000 (48%)]	Loss: 0.003833
Train Epoch: 14 [29440/60000 (49%)]	Loss: 0.013458
Train Epoch: 14 [30080/60000 (50%)]	Loss: 0.005005
Train Epoch: 14 [30720/60000 (51%)]	Loss: 0.007368
Train Epoch: 14 [31360/60000 (52%)]	Loss: 0.046243
Train Epoch: 14 [32000/60000 (53%)]	Loss: 0.019229
Train Epoch: 14 [32640/60000 (54%)]	Loss: 0.006932
Train Epoch: 14 [33280/60000 (55%)]	Loss: 0.018923
Train Epoch: 14 [33920/60000 (57%)]	Loss: 0.017587
Train Epoch: 14 [34560/60000 (58%)]	Loss: 0.000746
Train Epoch: 14 [35200/60000 (59%)]	Loss: 0.102284
Train Epoch: 14 [35840/60000 (60%)]	Loss: 0.023384
Train Epoch: 14 [36480/60000 (61%)]	Loss: 0.005457
Train Epoch: 14 [37120/60000 (62%)]	Loss: 0.006157
Train Epoch: 14 [37760/60000 (63%)]	Loss: 0.031551
Train Epoch: 14 [38400/60000 (64%)]	Loss: 0.074524
Train Epoch: 14 [39040/60000 (65%)]	Loss: 0.000644
Train Epoch: 14 [39680/60000 (66%)]	Loss: 0.012539
Train Epoch: 14 [40320/60000 (67%)]	Loss: 0.001708
Train Epoch: 14 [40960/60000 (6

Train Epoch: 14 [53120/60000 (88%)]	Loss: 0.019469
Train Epoch: 14 [53760/60000 (90%)]	Loss: 0.059504
Train Epoch: 14 [54400/60000 (91%)]	Loss: 0.031320
Train Epoch: 14 [55040/60000 (92%)]	Loss: 0.004289
Train Epoch: 14 [55680/60000 (93%)]	Loss: 0.062647
Train Epoch: 14 [56320/60000 (94%)]	Loss: 0.004157
Train Epoch: 14 [56960/60000 (95%)]	Loss: 0.003114
Train Epoch: 14 [57600/60000 (96%)]	Loss: 0.019688
Train Epoch: 14 [58240/60000 (97%)]	Loss: 0.002009
Train Epoch: 14 [58880/60000 (98%)]	Loss: 0.000172
Train Epoch: 14 [59520/60000 (99%)]	Loss: 0.003201

Test set: Average loss: 0.0286, Accuracy: 9917/10000 (99%)

[rank0]:[W619 20:48:00.670466910 ProcessGroupNCCL.cpp:1496] Warning: WARNING: destroy_process_group() was not called before program exit, which can leak resources. For more info, please see https://pytorch.org/docs/stable/distributed.html#shutdown (function operator())


2025-06-19 20:48:06,908	INFO	( 3 min) operation 833c9385-39d5f04a-24dd03e8-b3861697 completed


RunInfo(operation_spec={'intermediate_data_medium': 'nvme', 'intermediate_data_account': 'equal_amethyst_vulture', 'started_by': {'hostname': 'end-end-4.exec-nodes-end.nebius-playground.svc.kyt.k8s.nebius.yt', 'pid': 618, 'wrapper_version': '0.13.28', 'python_version': '3.12.10', 'binary_name': 'ipykernel_launcher.py', 'command': ['/slot/sandbox/jlab/lib/python42/site-packages/ipykernel_launcher.py', '-f', '/slot/sandbox/.local/share/jupyter/runtime/kernel-2a501a16-d03c-4234-9f78-97ef00f8ebd3.json'], 'user': 'root', 'platform': 'Debian GNU/Linux 12 (bookworm)'}, 'fail_on_job_restart': True, 'is_gang': True, 'annotations': {'is_tractorun': True}, 'tasks': {'task': {'command': 'python3 _py_runner.py wrapped.pickle config_dump _modules_info modules/_main_module.py _main_module PY_SOURCE', 'job_count': 1, 'gpu_limit': 1, 'port_count': 1, 'cpu_limit': 10, 'memory_limit': 32213319843, 'docker_image': 'cr.eu-north1.nebius.cloud/e00faee7vas5hpsh3s/solutions/examples:v5', 'file_paths': [{'value

## Tractorun cli

Let's consider a production-like scenario for running model training through the `Tractorun CLI`.

The `Tractorun CLI` allows:
1. Make model training reproducible.
2. Separating the model training code from the training run parameters. `Tractorun CLI` enables configuring the training process via a configuration file and CLI parameters.
3. Running the training module from any host where Python and `Tractorun` are installed.

We will use the official PyTorch [MNIST training example](https://github.com/pytorch/examples/blob/cdef4d43fb1a2c6c4349daa5080e4e8731c34569/mnist/main.py) again.
How to modify it with minimal changes to run using Tractorun:
1. Add `toolbox = prepare_and_get_toolbox(backend=Tractorch())` to the main function. Toolbox object provides useful utils for training like checkpoint manager, coordination metadata, initialized ytsaurus client, and more.
2. Add `file=sys.stderr` to each print.
3. Use `YtTensorDataset` instead of default `torch.Dataset`.

<details>
  <summary>Show the full diff</summary>

```diff
@@ -6,6 +6,15 @@
 from torchvision import datasets, transforms
 from torch.optim.lr_scheduler import StepLR

+import sys
+from tractorun.backend.tractorch import YtTensorDataset, Tractorch
+from tractorun.toolbox import Toolbox
+from tractorun.run import run
+from tractorun.mesh import Mesh
+from tractorun.resources import Resources
+from tractorun.stderr_reader import StderrMode
+from tractorun.backend.tractorch.serializer import TensorSerializer
+from tractorun.run import prepare_and_get_toolbox

 class Net(nn.Module):
     def __init__(self):
@@ -45,7 +54,7 @@
         if batch_idx % args.log_interval == 0:
             print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                 epoch, batch_idx * len(data), len(train_loader.dataset),
-                100. * batch_idx / len(train_loader), loss.item()))
+                100. * batch_idx / len(train_loader), loss.item()), file=sys.stderr)
             if args.dry_run:
                 break

@@ -66,10 +75,13 @@

     print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
         test_loss, correct, len(test_loader.dataset),
-        100. * correct / len(test_loader.dataset)))
+        100. * correct / len(test_loader.dataset)), file=sys.stderr)


 def main():
+    toolbox = prepare_and_get_toolbox(backend=Tractorch())
+    user_config = toolbox.get_user_config()
+
     # Training settings
     parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
     parser.add_argument('--batch-size', type=int, default=64, metavar='N',
@@ -94,7 +106,7 @@
                         help='how many batches to wait before logging training status')
     parser.add_argument('--save-model', action='store_true', default=False,
                         help='For Saving the current Model')
-    args = parser.parse_args()
+    args = parser.parse_args([])
     use_cuda = not args.no_cuda and torch.cuda.is_available()
     use_mps = not args.no_mps and torch.backends.mps.is_available()

@@ -120,10 +132,9 @@
         transforms.ToTensor(),
         transforms.Normalize((0.1307,), (0.3081,))
         ])
-    dataset1 = datasets.MNIST('../data', train=True, download=True,
-                       transform=transform)
-    dataset2 = datasets.MNIST('../data', train=False,
-                       transform=transform)
+    dataset1 = YtTensorDataset(toolbox=toolbox, path=user_config["dataset_train_path"], columns=['data', 'labels'])
+    dataset2 = YtTensorDataset(toolbox=toolbox, path=user_config["dataset_test_path"], columns=['data', 'labels'])
+
     train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
     test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

@@ -137,9 +148,9 @@
         scheduler.step()

     if args.save_model:
-        torch.save(model.state_dict(), "mnist_cnn.pt")
+        ts = TensorSerializer()
+        toolbox.save_model(ts.serialize(model.state_dict()), dataset_train_path, metadata={})


-if __name__ == '__main__':
+if __name__ == "__main__":
     main()
```
</details>

Let's create two files:
1. `main.py` with our model-training code.
2. `run_config.yaml` with training configuration.

In [19]:
code = r"""
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

import sys
from tractorun.backend.tractorch import YtTensorDataset, Tractorch
from tractorun.toolbox import Toolbox
from tractorun.run import run
from tractorun.mesh import Mesh
from tractorun.resources import Resources
from tractorun.stderr_reader import StderrMode
from tractorun.backend.tractorch.serializer import TensorSerializer
from tractorun.run import prepare_and_get_toolbox

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output


def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()), file=sys.stderr)
            if args.dry_run:
                break


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)), file=sys.stderr)


def main():
    toolbox = prepare_and_get_toolbox(backend=Tractorch())
    user_config = toolbox.get_user_config()

    # Training settings
    parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
    parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                        help='input batch size for training (default: 64)')
    parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
                        help='input batch size for testing (default: 1000)')
    parser.add_argument('--epochs', type=int, default=14, metavar='N',
                        help='number of epochs to train (default: 14)')
    parser.add_argument('--lr', type=float, default=1.0, metavar='LR',
                        help='learning rate (default: 1.0)')
    parser.add_argument('--gamma', type=float, default=0.7, metavar='M',
                        help='Learning rate step gamma (default: 0.7)')
    parser.add_argument('--no-cuda', action='store_true', default=False,
                        help='disables CUDA training')
    parser.add_argument('--no-mps', action='store_true', default=False,
                        help='disables macOS GPU training')
    parser.add_argument('--dry-run', action='store_true', default=False,
                        help='quickly check a single pass')
    parser.add_argument('--seed', type=int, default=1, metavar='S',
                        help='random seed (default: 1)')
    parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                        help='how many batches to wait before logging training status')
    parser.add_argument('--save-model', action='store_true', default=False,
                        help='For Saving the current Model')
    args = parser.parse_args([])
    use_cuda = not args.no_cuda and torch.cuda.is_available()
    use_mps = not args.no_mps and torch.backends.mps.is_available()

    torch.manual_seed(args.seed)

    if use_cuda:
        device = torch.device("cuda")
    elif use_mps:
        device = torch.device("mps")
    else:
        device = torch.device("cpu")

    train_kwargs = {'batch_size': args.batch_size}
    test_kwargs = {'batch_size': args.test_batch_size}
    if use_cuda:
        cuda_kwargs = {'num_workers': 1,
                       'pin_memory': True,
                       'shuffle': True}
        train_kwargs.update(cuda_kwargs)
        test_kwargs.update(cuda_kwargs)

    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])
    dataset1 = YtTensorDataset(toolbox=toolbox, yt_client=toolbox.yt_client, path=user_config["dataset_train_path"], columns=['data', 'labels'])
    dataset2 = YtTensorDataset(toolbox=toolbox, yt_client=toolbox.yt_client, path=user_config["dataset_test_path"], columns=['data', 'labels'])

    train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
    test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

    model = Net().to(device)
    optimizer = optim.Adadelta(model.parameters(), lr=args.lr)

    scheduler = StepLR(optimizer, step_size=1, gamma=args.gamma)
    for epoch in range(1, args.epochs + 1):
        train(args, model, device, train_loader, optimizer, epoch)
        test(model, device, test_loader)
        scheduler.step()

    if args.save_model:
        ts = TensorSerializer()
        toolbox.save_model(ts.serialize(model.state_dict()), dataset_train_path, metadata={})


if __name__ == "__main__":
    main()
"""

with open("main.py", "w") as f:
    f.write(code)

In [20]:
import yaml


config = {
    "yt_path": training_dir,
    "mesh": {
        "node_count": 1,
        "process_per_node": 1,
        "gpu_per_process": 0,
        "pool_trees": list(map(str, h100_pool_trees)),
    },  
    "user_config": {
        "dataset_train_path": dataset_train_path,
        "dataset_test_path": dataset_test_path,
    },
    "resources": {
        "cpu_limit": 10,
        "memory_limit": 32212254720,
    },
    "bind_local": [
        {
            "source": "./main.py",
            "destination": "/tractorun/main.py",
        },
    ],
    "command": ["python3", "/tractorun/main.py"],
    "proxy_stderr_mode": "primary",
}

with open("run_config.yaml", "w") as f:
    yaml.dump(config, f)

In [21]:
!tractorun --run-config-path run_config.yaml

Starting upload:   0%|                                              | 0.00/19.6k

(1/2) [UPLOAD] __file_0.zip:   0%|                                  | 0.00/19.6k

(1/2) [UPLOAD] __file_0.zip:  33%|##########7                      | 6.39k/19.6k

(1/2) [OK] __file_0.zip:  33%|############                         | 6.39k/19.6k

(1/2) [OK] __file_0.zip:  33%|############                         | 6.39k/19.6k

...

Train Epoch: 14 [22400/60000 (37%)]	Loss: 0.006572
Train Epoch: 14 [23040/60000 (38%)]	Loss: 0.006415
Train Epoch: 14 [23680/60000 (39%)]	Loss: 0.016754
Train Epoch: 14 [24320/60000 (41%)]	Loss: 0.000284
Train Epoch: 14 [24960/60000 (42%)]	Loss: 0.000512
Train Epoch: 14 [25600/60000 (43%)]	Loss: 0.005908
Train Epoch: 14 [26240/60000 (44%)]	Loss: 0.010500
Train Epoch: 14 [26880/60000 (45%)]	Loss: 0.015310
Train Epoch: 14 [27520/60000 (46%)]	Loss: 0.020943
Train Epoch: 14 [28160/60000 (47%)]	Loss: 0.007476
Train Epoch: 14 [28800/60000 (48%)]	Loss: 0.001789
Train Epoch: 14 [29440/60000 (49%)]	Loss: 0.026523
Train Epoch: 14 [30080/60000 (50%)]	Loss: 0.022921
Train Epoch: 14 [30720/60000 (51%)]	Loss: 0.004790
Train Epoch: 14 [31360/60000 (52%)]	Loss: 0.025908
Train Epoch: 14 [32000/60000 (53%)]	Loss: 0.015199
Train Epoch: 14 [32640/60000 (54%)]	Loss: 0.029967
Train Epoch: 14 [33280/60000 (55%)]	Loss: 0.026125
Train Epoch: 14 [33920/60000 (57%)]	Loss: 0.001464
Train Epoch: 14 [34560/60000 (5

Train Epoch: 14 [37120/60000 (62%)]	Loss: 0.009436
Train Epoch: 14 [37760/60000 (63%)]	Loss: 0.078857
Train Epoch: 14 [38400/60000 (64%)]	Loss: 0.042718
Train Epoch: 14 [39040/60000 (65%)]	Loss: 0.000777
Train Epoch: 14 [39680/60000 (66%)]	Loss: 0.042462
Train Epoch: 14 [40320/60000 (67%)]	Loss: 0.004990
Train Epoch: 14 [40960/60000 (68%)]	Loss: 0.011255
Train Epoch: 14 [41600/60000 (69%)]	Loss: 0.005602
Train Epoch: 14 [42240/60000 (70%)]	Loss: 0.008519
Train Epoch: 14 [42880/60000 (71%)]	Loss: 0.009623
Train Epoch: 14 [43520/60000 (72%)]	Loss: 0.011592
Train Epoch: 14 [44160/60000 (74%)]	Loss: 0.000449
Train Epoch: 14 [44800/60000 (75%)]	Loss: 0.009011
Train Epoch: 14 [45440/60000 (76%)]	Loss: 0.025084
Train Epoch: 14 [46080/60000 (77%)]	Loss: 0.020728
Train Epoch: 14 [46720/60000 (78%)]	Loss: 0.054390
Train Epoch: 14 [47360/60000 (79%)]	Loss: 0.012415
Train Epoch: 14 [48000/60000 (80%)]	Loss: 0.024724
Train Epoch: 14 [48640/60000 (81%)]	Loss: 0.007399
Train Epoch: 14 [49280/60000 (8

Train Epoch: 14 [51200/60000 (85%)]	Loss: 0.042542
Train Epoch: 14 [51840/60000 (86%)]	Loss: 0.002729
Train Epoch: 14 [52480/60000 (87%)]	Loss: 0.004932
Train Epoch: 14 [53120/60000 (88%)]	Loss: 0.000500
Train Epoch: 14 [53760/60000 (90%)]	Loss: 0.027206
Train Epoch: 14 [54400/60000 (91%)]	Loss: 0.002522
Train Epoch: 14 [55040/60000 (92%)]	Loss: 0.001117
Train Epoch: 14 [55680/60000 (93%)]	Loss: 0.045572
Train Epoch: 14 [56320/60000 (94%)]	Loss: 0.027302
Train Epoch: 14 [56960/60000 (95%)]	Loss: 0.018647
Train Epoch: 14 [57600/60000 (96%)]	Loss: 0.036808
Train Epoch: 14 [58240/60000 (97%)]	Loss: 0.000488
Train Epoch: 14 [58880/60000 (98%)]	Loss: 0.006383
Train Epoch: 14 [59520/60000 (99%)]	Loss: 0.000301



Test set: Average loss: 0.0267, Accuracy: 9915/10000 (99%)



2025-06-19 20:57:06,591	INFO	( 5 min) operation af3941d-fca6ac76-24dd03e8-9b60f6d2 completed
